In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.preprocessing import LabelEncoder

In [2]:
train_path = "data/train.csv"
test_path = "data/test.csv"
sample_path = "data/sample_submission.csv"
data_dict = "data/data_dictionary.csv"

train = pd.read_csv(train_path)

In [17]:
missing_indices = train[train["donor_age"].isna()].index
random_samples = np.random.choice(train["donor_age"].dropna(), size=len(missing_indices), replace=True)

# Assign random samples to the missing values
train.loc[missing_indices, "donor_age"] = random_samples

In [25]:
count = 0
for col in train.columns:
    if train[col].dtype != "object" and len(train[col].unique()) >= 100:
        count += 1
        print(f"{col}: {len(train[col].unique())}")

print(count)

ID: 28800
donor_age: 20909
age_at_hct: 22168
efs_time: 19208
4


In [26]:
count = 0
for col in train.columns:
    if train[col].dtype == "object":
        # if 'Not done' in train[col].unique():
        #     print(f"{col}: Not done")
        #     train[col] = train[col].fillna('Not Done')
        if 'Not tested' in train[col].unique():
            train[col] = train[col].fillna('Not Done')
        count += 1
        print(f"{col}: {train[col].unique()}")

print(count)

dri_score: ['N/A - non-malignant indication' 'Intermediate' 'High' 'Low'
 'N/A - disease not classifiable' 'N/A - pediatric' 'TBD cytogenetics'
 'Intermediate - TED AML case <missing cytogenetics' nan
 'High - TED AML case <missing cytogenetics' 'Very high'
 'Missing disease status']
psych_disturb: ['No' 'Not done' 'Yes' nan]
cyto_score: [nan 'Intermediate' 'Poor' 'Other' 'Favorable' 'TBD' 'Normal' 'Not tested']
diabetes: ['No' 'Yes' 'Not done' nan]
tbi_status: ['No TBI' 'TBI +- Other, >cGy' 'TBI + Cy +- Other' 'TBI +- Other, <=cGy'
 'TBI +- Other, unknown dose' 'TBI +- Other, -cGy, fractionated'
 'TBI +- Other, -cGy, single' 'TBI +- Other, -cGy, unknown dose']
arrhythmia: ['No' 'Yes' nan 'Not done']
graft_type: ['Bone marrow' 'Peripheral blood']
vent_hist: ['No' 'Yes' nan]
renal_issue: ['No' nan 'Yes' 'Not done']
pulm_severe: ['No' 'Yes' nan 'Not done']
prim_disease_hct: ['IEA' 'AML' 'HIS' 'ALL' 'MPN' 'IIS' 'Solid tumor' 'Other leukemia' 'PCD'
 'IPA' 'IMD' 'MDS' 'NHL' 'SAA' 'AI' 'CML'